In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,095 kB]
Get:15 htt

In [ ]:
# Start Spark session
# download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Spark session with an additional option that adds the driver to Spark:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big_Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

--2020-10-01 06:04:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-10-01 06:04:05 (9.48 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz'

In [ ]:
spark.sparkContext.addFile(url)
sport_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Sports_v1_00.tsv.gz'), sep='\t', header=True)
sport_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
sport_df.count()


4850360

In [ ]:
sport_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [ ]:
sport_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
sport_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [ ]:
sport_df.describe().show()

+-------+-----------+--------------------+--------------+-------------------+-------------------+----------------------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|         product_id|     product_parent|                     product_title|product_category|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+-------------------+-------------------+----------------------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|  count|    4850360|             4850360|       4850360|            4850360|            4850360|                           4850360|       

In [ ]:
clean_sport_df = sport_df.dropna().dropDuplicates()
clean_sport_df.count()

4849000

review ID table has:

> review_id TEXT PRIMARY KEY NOT NULL,  
customer_id INTEGER,  
product_id TEXT,  
product_parent INTEGER,  
review_date DATE -- this should be in the formate yyyy-mm-dd

product table has:
>   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,  product_title TEXT

customer table has:
>   customer_id INT PRIMARY KEY NOT NULL UNIQUE,  customer_count INT

vine table has:
>   review_id TEXT PRIMARY KEY,  
star_rating INTEGER,  
helpful_votes INTEGER,  
total_votes INTEGER,  
vine TEXT





In [ ]:
# Import types and col function
from pyspark.sql.types import StringType, IntegerType, DateType
from pyspark.sql.functions import col

In [ ]:
# create review table
review_df = clean_sport_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_df = review_df.withColumn("review_id",col("review_id").cast(StringType()))\
  .withColumn("customer_id",col("customer_id").cast(IntegerType()))\
  .withColumn("product_id",col("product_id").cast(StringType()))\
  .withColumn("product_parent",col("product_parent").cast(IntegerType()))\
  .withColumn("review_date",col("review_date").cast(DateType()))
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1008599CB6XOO|   24002988|B005J9TKZ8|     807909508| 2014-11-27|
|R100IR1LJNAK0E|   14266788|B002IWXJ4W|     632337843| 2013-04-02|
|R100JTP9E01W14|   34674746|B006CM9Q3M|     476438308| 2013-01-04|
|R100S0IZLWJDS4|   31805125|B004000CCW|      85717718| 2011-08-18|
| R100U3Q12G58D|   26626320|B00B5P22JG|     437589404| 2015-08-20|
|R1013PCJ63HJMO|   51754455|B002M7871I|     883570298| 2012-01-30|
|R101AE0OGK16A7|   14279604|B000BUC7IM|     811197073| 2012-06-07|
|R101F7X1H5593W|   23726987|B004R9V6S0|     416992541| 2012-11-12|
|R101RXI99MOWQY|   29547051|B00004NKIQ|     147383064| 2015-05-25|
|R101Z6EEEJCEDL|   13342689|B00KJLDP98|     103323654| 2014-10-31|
|R1023SWPQVRTYV|   10388347|B0009TNGQC|      64212709| 2013-04-06|
|R1025WVI1PTVB0|    1718722|B0030BEBA6|     770842719| 2015-05

In [ ]:
# create poduct table
product_df = clean_sport_df.select(['product_id', 'product_title'])
cleaned_product_df = product_df.dropDuplicates(["product_id"])
# Set columns types
cleaned_product_df = cleaned_product_df.withColumn("product_id",col("product_id").cast(StringType()))\
  .withColumn("product_title",col("product_title").cast(StringType()))
cleaned_product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005J9TKZ8|     NCAA Pillow Pet|
|B002IWXJ4W|Magpul XTM Rail P...|
|B006CM9Q3M|Phoenix Technolog...|
|B004000CCW|mp5 pdw silencer ...|
|B00B5P22JG|Under Armour Men'...|
|B002M7871I|Mizuno Women's Ni...|
|B000BUC7IM|South Bend Monofi...|
|B004R9V6S0|Bushnell Tour V2 ...|
|B00004NKIQ|Franklin Sports A...|
|B00KJLDP98|Stainless Steel 3...|
|B0009TNGQC|Ace Martial Arts ...|
|B0030BEBA6|CW-X Women's Stab...|
|B009PAHH98|SharkMotorcycleAu...|
|B004J1FPWE|16 Inch Kids' Gyr...|
|B000VXFIU8|PowerBait Turbo D...|
|B000W1A836|DynaFlex TourGrip...|
|B004MY5L9U|Nikon Prostaff 3-...|
|B007RPXYP4|5 (FIVE) Excuse C...|
|B000N8OID4|Galco Waistband I...|
|B00IO5NNJ8|OEM Volvo Penta M...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# create customer table
from pyspark.sql.functions import col
customer_df = clean_sport_df.groupBy('customer_id').count()
customer_df.sort(col('count')).show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    6060548|    1|
|   16872545|    1|
|   39872740|    1|
|   16254686|    1|
|   11040445|    1|
|   26655197|    1|
|   52960291|    1|
|   16303700|    1|
|   27293091|    1|
|    1171167|    1|
|   19946204|    1|
|   29412283|    1|
|   34593885|    1|
|   36923466|    1|
|   28234212|    1|
|   11395818|    1|
|     498279|    1|
|   15087244|    1|
|   38824581|    1|
|   15313678|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
# change 'count' column name to 'customer_count' and sort
customer_df = customer_df.withColumnRenamed('count', 'customer_count')
customer_df = customer_df.dropDuplicates(["customer_id"])
# Set columns types
customer_df = customer_df.withColumn("customer_id",col("customer_id").cast(IntegerType()))\
  .withColumn("customer_count",col("customer_count").cast(IntegerType()))
customer_df = customer_df.sort(col('count'), ascending = False)
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50820654|           552|
|   37651511|           294|
|   26955164|           272|
|   50227539|           257|
|   52870270|           245|
|   25355517|           225|
|   13355404|           193|
|   46160224|           177|
|   26772822|           172|
|   52770861|           170|
|   49335121|           162|
|    1871743|           154|
|   25436988|           144|
|   20032310|           143|
|   19512365|           142|
|   51354368|           140|
|   18453567|           138|
|   29505740|           136|
|   36579538|           136|
|   52571463|           132|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customer_df.count()

2817835

In [ ]:
# prepare to validate vine reviews by filtering for only verified purchases 
# to help filter out fake reviews
vine_verified_df = sport_df.filter(sport_df['verified_purchase'] == 'Y')
vine_verified_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|            1|          1|   N|                Y|       Shipped fast.|These are the bes...| 2015-08-31|
|         US|   45813853| RR8V7WR27NXJ5|B008VS8M58|     962442336|Baoer 223 5.56x45...|          Sports|          1|    

In [ ]:
vine_verified_df.count()

4282978

In [ ]:
clean_vine_df = vine_verified_df.dropna().dropDuplicates()
clean_vine_df.count()

4282482

In [ ]:
# create vine table for DB
vine_df = clean_vine_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1008599CB6XOO|          5|            0|          0|   N|
|R100IR1LJNAK0E|          5|            0|          0|   N|
|R100JTP9E01W14|          4|            1|          1|   N|
|R100S0IZLWJDS4|          1|            5|          6|   N|
| R100U3Q12G58D|          1|            0|          0|   N|
|R1013PCJ63HJMO|          1|            2|          2|   N|
|R101AE0OGK16A7|          5|            2|          2|   N|
|R101F7X1H5593W|          5|            0|          0|   N|
|R101RXI99MOWQY|          5|            0|          0|   N|
|R1023SWPQVRTYV|          4|            0|          0|   N|
|R1025WVI1PTVB0|          5|            1|          1|   N|
|R102645XLOMNKM|          1|            2|          2|   N|
|R1027OX6YNKJGP|          5|            3|          3|   N|
|R102D411NGWB8M|          5|            

In [ ]:
vine_df.count()

4282482

In [ ]:
# create DF with review lengths to help filter out fake reviews
vine_reviews_df = clean_vine_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'review_body'])
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|
+--------------+-----------+-------------+-----------+----+--------------------+
|R1008599CB6XOO|          5|            0|          0|   N|    This gift rocks!|
|R100IR1LJNAK0E|          5|            0|          0|   N|Great<br />  Well...|
|R100JTP9E01W14|          4|            1|          1|   N|Like others said ...|
|R100S0IZLWJDS4|          1|            5|          6|   N|We bought this gu...|
| R100U3Q12G58D|          1|            0|          0|   N|Size is at least ...|
|R1013PCJ63HJMO|          1|            2|          2|   N|These pants are t...|
|R101AE0OGK16A7|          5|            2|          2|   N|Sometime even the...|
|R101F7X1H5593W|          5|            0|          0|   N|Hello - I bought ...|
|R101RXI99MOWQY|          5|            0|          0|   N|My 9 year old son...|
|R1023SWPQVRTYV|          4|

In [ ]:
# select only Vine reviews
vine_filter_df = vine_reviews_df.filter(vine_reviews_df['vine'] == 'Y')
vine_filter_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|
+--------------+-----------+-------------+-----------+----+--------------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|The handy multi-p...|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|This is a great p...|
|R3FYZFOAA7EZU9|          5|            1|          1|   Y|Before using Srix...|
|R20N8XGNA7X5EB|          5|

In [ ]:
# only looking at Vine reviews
vine_filter_df.count()

11

In [ ]:
# analyze vine table
# number of review
# average rating
# number of helpful votes
# count word length of vine reviews, bin, distribution
from pyspark.sql.functions import mean, stddev
# vine_mean = vine_filter_df.select(mean(col('star_rating')))[0]
vine_mean = vine_filter_df.select(mean(col('star_rating'))).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|4.818181818181818|
+-----------------+



In [ ]:
# review lengths can be an indicator of veracity; spoofed reviews tend to be shorter
from pyspark.sql.functions import length
# Create a length column to be used as a future feature
review_length_df = vine_filter_df.withColumn('length', length(vine_filter_df['review_body']))
review_length_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|
+--------------+-----------+-------------+-----------+----+--------------------+------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|The handy multi-p...|  5493|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|This is a great p...|   888|
|R3FYZFOAA7EZU9|          5|    

In [ ]:
review_length_df.describe().show()

+-------+--------------+-------------------+------------------+------------------+----+--------------------+------------------+
|summary|     review_id|        star_rating|     helpful_votes|       total_votes|vine|         review_body|            length|
+-------+--------------+-------------------+------------------+------------------+----+--------------------+------------------+
|  count|            11|                 11|                11|                11|  11|                  11|                11|
|   mean|          null|  4.818181818181818|               5.0|5.7272727272727275|null|                null|1290.1818181818182|
| stddev|          null|0.40451991747794536|12.672805529952713|14.402020060331182|null|                null|1550.5555661234341|
|    min|R1H8QU9G9HLBWK|                  4|                 0|                 0|   Y|Almost everyone i...|               265|
|    max| RUVM52QIHJ57R|                  5|                43|                49|   Y|This is a great p

In [ ]:
# Add new column to hold word counts > 20
# typical rules on websites are to have reviews longer than 20 words
review_length_df.withColumn('above_avg_length', review_length_df['length']>20).show()

+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|above_avg_length|
+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|            true|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|            true|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|            true|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|            true|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|            true|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|            true|
|R1H8QU9G9HLBWK|          5|           43|         49| 

In [ ]:
# see number of reviews with greater than 20 words
review_length_df.filter(review_length_df.length >20).count()

11

In [ ]:
long_reviews = review_length_df.filter(review_length_df.length >20).count()
total_reviews = review_length_df.count()

print(long_reviews/total_reviews)

1.0


In [ ]:
# Add new column to hold word counts > mean
review_length_df.withColumn('above_avg_length', review_length_df['length']>1290).show()

+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|above_avg_length|
+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|           false|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|           false|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|           false|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|            true|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|           false|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|           false|
|R1H8QU9G9HLBWK|          5|           43|         49| 

In [ ]:
# See number of reviews that are above average in length
review_above_mean = review_length_df.filter(review_length_df.length > 1290).count()
print(review_above_mean)

3


In [ ]:
vine_final_df = review_length_df.drop('review_body', 'length')
# Set columns types
vine_final_df = vine_final_df.withColumn("review_id",col("review_id").cast(StringType()))\
  .withColumn("star_rating",col("star_rating").cast(IntegerType()))\
  .withColumn("helpful_votes",col("helpful_votes").cast(IntegerType()))\
  .withColumn("total_votes",col("total_votes").cast(IntegerType()))\
  .withColumn("vine",col("vine").cast(StringType()))
vine_final_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|
| RUVM52QIHJ57R|          5|            1|          1|   Y|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|
|R3FYZFOAA7EZU9|          5|            1|          1|   Y|
|R20N8XGNA7X5EB|          5|            0|          0|   Y|
|R2FCLG4I9PIQSO|          4|            4|          4|   Y|
+--------------+-----------+-------------+-----------+----+



In [ ]:
#  use pySpark to load data into AWS S3

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://aws.czi9q1pcbluc.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": " ---- ", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to reviews table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
# Write DataFrame to products table in RDS
cleaned_product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
# Write DataFrame to customers table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
# Write DataFrame to Vine reviews table in RDS
vine_final_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Evaluation of the Vine reviews looked at the total number of reviews, average rating, number of votes and length of reviews.  The Vine reviews were removed from the total review population and examined separately.

The length of reviews is the strongest indicator that a review is legitimate and trustworthy.  Second to this is the number of helpful votes.

Since this is such small result of Vine reviews for this category, further analysis isn't justified in this case, trying to see the distribution or look for outliers is not informative with a population size smaller than typical sample sizes.

Of all of the sports product reviews in this dataset, there are only 11 vine reviews, three of which have word counts above the mean.  Spoofed or SPAM reviews are usually shorter in length.  Some websites use rules of thumb like at least 20 words for a review to be accepted.  All of these reviews exceeded that threshold by an order of magnitude.  Furthermore, the shortest review was 265 words.  A typed, full-length page has approximately 250 words.  This is quite lengthy for a review and most any that are this long or longer are likely to be trustworthy reviews.

You can say these sports product fans aren't just fans of the product; they're fanatical about their reviews as well.
